In [1]:
import torch
from torchvision.models import resnet18
import torch_pruning as tp
from yolox.models import yolox_tiny



model = yolox_tiny().cuda()
#model = torch.load('/workspace/tensorrt/model.pth')
print(model)
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

/usr/local/lib/python3.8/dist-packages/tqdm-4.64.0-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [2]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [3]:
'''
temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2[1])
for i in range(len(temp2)):
    if 'backbone.backbone' in temp2[i]:
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝
'''

"\ntemp = []\ntemp2 = list(model.state_dict().keys())\nnum_list = [0,1,2,3]\n#print(temp2[1])\nfor i in range(len(temp2)):\n    if 'backbone.backbone' in temp2[i]:\n        if 'weight' in temp2[i]:\n            temp2[i] = temp2[i].replace('2.0','2[0]')\n            temp2[i] = temp2[i].replace('2.1','2[1]')\n            temp2[i] = temp2[i].replace('3.0','3[0]')\n            temp2[i] = temp2[i].replace('3.1','3[1]')\n            temp2[i] = temp2[i].replace('4.0','4[0]')\n            temp2[i] = temp2[i].replace('4.1','4[1]')\n            temp2[i] = temp2[i].replace('5.0','5[0]')\n            temp2[i] = temp2[i].replace('5.1','5[1]')\n            temp2[i] = temp2[i].replace('5.2','5[2]')\n            temp2[i] = temp2[i].replace('m.0','m[0]')\n            temp2[i] = temp2[i].replace('m.1','m[1]')\n            temp2[i] = temp2[i].replace('m.2','m[2]')\n            temp.append(temp2[i])\n\nprint(temp)\n# backbone 만 프루닝\n"

In [4]:

pruning_idxs = strategy(model.backbone.backbone.dark3[0].conv.weight, amount=0.2) # or pruning_idxs=[2, 6, 9, ...]
print(pruning_idxs)
pruning_plan = DG.get_pruning_plan(model.backbone.backbone.dark3[0].conv ,tp.prune_conv_out_channel, idxs=pruning_idxs )
print(pruning_plan)

    # 4. execute this plan after checking (prune the model)
    #    if the plan prunes some channels to zero, 
    #    DG.check_pruning plan will return False.
if DG.check_pruning_plan(pruning_plan):
    pruning_plan.exec()
        
        
        
torch.save(model, 'model.pth') 
model = yolox_tiny().cuda()
print(model)

torch.load('/workspace/tensorrt/model.pth')



[7, 14, 24, 35, 36, 40, 41, 50, 51, 57, 58, 61, 69, 70, 74, 78, 80, 82, 94]

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.0.conv (Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.0.conv (Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)), Index=[7, 14, 24, 35, 36, 40, 41, 50, 51, 57, 58, 61, 69, 70, 74, 78, 80, 82, 94], metric={'#params': 8208}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.0.conv (Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.0.bn (BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[7, 14, 24, 35, 36, 40, 41, 50, 51, 57, 58, 61, 69, 70, 74, 78, 80, 82, 94], metric={'#params': 38}]
[ [DEP] BatchnormPruner on

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [5]:
temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2[1])
for i in range(len(temp2)):
    if 'backbone.backbone' in temp2[i]:
        if 'weight' in temp2[i]:
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝

['backbone.backbone.stem.conv.conv.weight', 'backbone.backbone.stem.conv.bn.weight', 'backbone.backbone.dark2.0.conv.weight', 'backbone.backbone.dark2.0.bn.weight', 'backbone.backbone.dark2.1.conv1.conv.weight', 'backbone.backbone.dark2.1.conv1.bn.weight', 'backbone.backbone.dark2.1.conv2.conv.weight', 'backbone.backbone.dark2.1.conv2.bn.weight', 'backbone.backbone.dark2.1.conv3.conv.weight', 'backbone.backbone.dark2.1.conv3.bn.weight', 'backbone.backbone.dark2.1.m.0.conv1.conv.weight', 'backbone.backbone.dark2.1.m.0.conv1.bn.weight', 'backbone.backbone.dark2.1.m.0.conv2.conv.weight', 'backbone.backbone.dark2.1.m.0.conv2.bn.weight', 'backbone.backbone.dark3.0.conv.weight', 'backbone.backbone.dark3.0.bn.weight', 'backbone.backbone.dark3.1.conv1.conv.weight', 'backbone.backbone.dark3.1.conv1.bn.weight', 'backbone.backbone.dark3.1.conv2.conv.weight', 'backbone.backbone.dark3.1.conv2.bn.weight', 'backbone.backbone.dark3.1.conv3.conv.weight', 'backbone.backbone.dark3.1.conv3.bn.weight', 'ba